In [1]:
from dotenv import load_dotenv
import os
import base64
from io import StringIO
import snowflake.connector as sf
from dotenv import load_dotenv
import json
import pandas as pd
load_dotenv()
from e2b_code_interpreter import Sandbox

In [2]:
def sf_client():
    conn = sf.connect(
    user=os.getenv('SF_USER'),
    password=os.getenv('SF_PASSWORD'),
    account=os.getenv('SF_ACCOUNT'),
    warehouse=os.getenv('SF_WAREHOUSE'),
    database=os.getenv('SF_DATABASE'),
    schema=os.getenv('SF_SCHEMA'),
    role='FRED_ROLE',
    private_key_file = 'rsa_key.p8'
    )
    return conn
 
conn = sf_client()
cursor = conn.cursor()

In [3]:
df = cursor.execute('select * from STOCK_DATA').fetch_pandas_all()

In [4]:
os.makedirs('local', exist_ok=True)
df.to_csv('local/data.csv', index=False)

### LLM SQL Generation

In [5]:
from utils.snowflake.core import write_to_csv
from utils.helper import sql_query_generation_prompt
from utils.litellm.core import llm
import json 

result = llm(model='gemini/gemini-1.5-pro', system_prompt=sql_query_generation_prompt, user_prompt='generate 5 sql queries', is_json=True)
print(result)
answer = json.loads(result['answer']) if isinstance(result['answer'], str) else result['answer']

{'id': 'chatcmpl-605e0766-1e7b-4578-99e6-648874233eb5', 'prompt': 'generate 5 sql queries', 'answer': '[{"Title": "30-Day Moving Average", "SQL": "SELECT DATA_DATE, AVG(VALUE) OVER (ORDER BY DATA_DATE ASC ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS MOVING_AVERAGE_30 FROM STOCK_DATA", "Description": "This query calculates the 30-day moving average of the S&P 500, smoothing out short-term volatility and highlighting longer-term trends."}, {"Title": "Year-Over-Year Growth", "SQL": "SELECT DATA_DATE, VALUE, LAG(VALUE, 365) OVER (ORDER BY DATA_DATE) AS LAST_YEAR_VALUE, (VALUE - LAG(VALUE, 365) OVER (ORDER BY DATA_DATE)) * 100.0 / LAG(VALUE, 365) OVER (ORDER BY DATA_DATE) AS YOY_GROWTH FROM STOCK_DATA", "Description": "This query calculates the year-over-year growth percentage, showing how the current value compares to the same date in the previous year."}, {"Title": "Rolling 90-Day Volatility", "SQL": "SELECT DATA_DATE, STDDEV(VALUE) OVER (ORDER BY DATA_DATE ASC ROWS BETWEEN 89 PRECEDING 

In [10]:
with open('local/charts.json', 'w') as f:
    f.write(json.dumps(answer))

In [ ]:
# sqls = []
# for _ in answer:
#     sqls.append(_['SQL']+';')
# with open('local/charts.sql', 'w') as f:
#     f.write("\n".join(sqls))

### SQL (Snowflake) -> CSV

In [ ]:
# from utils.snowflake.core import write_to_csv

# with open('local/charts.sql', 'r') as f:
#     sql_content = f.read()
    
# for idx, sql in enumerate(sql_content.split('\n')):
#     write_to_csv(sql, idx )
    

In [8]:
import pandas as pd
df = pd.read_csv('local/3.csv', parse_dates=['DATA_DATE'])
df.head(5).to_string()

'   DATA_DATE    VALUE  MOVING_AVERAGE_5\n0 2020-03-30  2626.65          2626.650\n1 2020-03-31  2584.59          2605.620\n2 2020-04-01  2470.50          2560.580\n3 2020-04-02  2526.90          2552.160\n4 2020-04-03  2488.65          2539.458'

In [16]:
from e2b_code_interpreter import Sandbox
import base64
import json
import pandas as pd
from utils.litellm.core import llm
from utils.helper import sql_query_generation_prompt, python_code_generation_prompt
from utils.s3.core import upload_png_to_s3, get_s3_client
from utils.snowflake.core import write_to_csv
chart_data = []
with open('local/charts.json', 'r') as f:
    chart_metadata = json.loads(f.read())
for idx, _ in enumerate(chart_metadata):
    df = write_to_csv(_['SQL'].strip(';'))
    sbx = Sandbox()
    top_5_data = pd.read_csv('local/data.csv').head(5).to_string()
    with open("local/data.csv", "rb") as file:
        sbx.files.write("/home/user/sandbox/data.csv", file)
    result = llm(model='gemini/gemini-2.5-pro-exp-03-25', system_prompt=python_code_generation_prompt, user_prompt=top_5_data, is_json=True)['answer']
    code_to_run = json.loads(result)["code_to_run"] if isinstance(result,str) else result["code_to_run"]
    execution = sbx.run_code(code_to_run)
    print(execution)
    img_bytes = base64.b64decode(execution.results[0].text)
    img_url=upload_png_to_s3(get_s3_client(), 'charts',img_bytes)
    chart_data.append( {'title' : _['Title'], 'description' : _['Description'], 'chart_url': img_url } )

Execution(Results: [Result(...)], Logs: Logs(stdout: ['Collecting kaleido\n  Downloading kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)\n\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/79.9 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b\r\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.4/79.9 MB\x1b \x1b12.3 MB/s\x1b eta \x1b0:00:07\x1b\r\x1b[2K     \x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.1/79.9 MB\x1b \x1b15.9 MB/s\x1b eta \x1b0:00:05\x1b\r\x1b[2K     \x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.9/79.9 MB\x1b \x1b19.0 MB/s\x1b eta \x1b0:00:05\x1b', '\r\x1b[2K     \x1b━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b3.1/79.9 MB\x1b \x1b21.7 MB/s\x1b eta \x1b0:00:04\x1b\r\x1b[2K     \x1b━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b4.5/79.9 MB\x1b \x1b25.4 MB/s\x1b eta \x1b0:00:03\x1b\r\x1b[2K     \x1b━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b6.3/

In [18]:
with open('local/chart_output.json', 'w') as f:
    f.write(json.dumps(chart_data))

In [37]:
schema = {col: str(df[col].dtype) for col in df.columns}

In [ ]:
code_to_run = """
import subprocess

# Ensure Kaleido is installed
subprocess.run(["pip", "install", "kaleido"], check=True)

import plotly.express as px
import pandas as pd
import io
import base64

df = pd.read_csv('/home/user/sandbox/data.csv')
#NEVER CHANGE THE ABOVE LINES OF CODE, UNLESS EXTRA LIBRARIES ARE REQUIRED FOR INPORT

# Convert DATA_DATE to datetime format
df['DATA_DATE'] = pd.to_datetime(df['DATA_DATE'])

# Create the plot
fig = px.line(df, x='DATA_DATE', y='MOVING_AVERAGE_30', title='S&P 500 30-Day Moving Average')

#NEVER CHANGE THE BELOW LINES OF CODE
img_bytes = io.BytesIO()
fig.write_image(img_bytes, format="png")  # Requires kaleido
img_base64 = base64.b64encode(img_bytes.getvalue()).decode("utf-8")
img_base64
"""


In [19]:
from utils.sandbox.core import python_sandbox
with open('local/charts.json', 'r') as f:
    chart_metadata = json.loads(f.read())
chart_data = python_sandbox(chart_metadata)

Execution(Results: [Result(...)], Logs: Logs(stdout: ['Collecting kaleido\n  Downloading kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)\n\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/79.9 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b\r\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.3/79.9 MB\x1b \x1b10.0 MB/s\x1b eta \x1b0:00:08\x1b\r\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.9/79.9 MB\x1b \x1b13.3 MB/s\x1b eta \x1b0:00:06\x1b\r\x1b[2K     \x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.7/79.9 MB\x1b \x1b16.1 MB/s\x1b eta \x1b0:00:05\x1b', '\r\x1b[2K     \x1b━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b2.7/79.9 MB\x1b \x1b18.9 MB/s\x1b eta \x1b0:00:05\x1b\r\x1b[2K     \x1b━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b3.9/79.9 MB\x1b \x1b22.1 MB/s\x1b eta \x1b0:00:04\x1b\r\x1b[2K     \x1b━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b5.6/79.9 MB\

In [20]:
chart_data

[{'title': '30-Day Moving Average',
  'description': 'This query calculates the 30-day moving average of the S&P 500, smoothing out short-term volatility and highlighting longer-term trends.',
  'chart_url': 'https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/charts/af06dcda-46cd-43db-9b09-f0f56224390b.png'},
 {'title': 'Year-Over-Year Growth',
  'description': 'This query calculates the year-over-year growth percentage, showing how the current value compares to the same date in the previous year.',
  'chart_url': 'https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/charts/546f1e18-f970-4846-979c-45c892eddf5d.png'},
 {'title': 'Rolling 90-Day Volatility',
  'description': 'This query calculates the rolling 90-day standard deviation of the S&P 500, providing insights into market volatility over time. Higher values indicate greater price fluctuations within the 90-day window.',
  'chart_url': 'https://sfopenaccessbucket.s3.us-east-1.amazonaws.com/charts/d8d1b8a2-6ed1-42f4-bad6-54db1

In [27]:
with open('links.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [30]:
data['results']

[{'WEBPAGE_TITLE': 'S&P 500 Gains and Losses Today: Supermicro, Arista Networks, and Nvidia Lead Tech Selloff - Yahoo Finance',
  'WEBPAGE_URL': 'https://finance.yahoo.com/news/p-500-gains-losses-today-205835836.html',
  'WEBPAGE_CONTENT': "Finance New on Yahoo Yahoo Finance In This Article: The S&P 500 fell 1.1% on Wednesday, March 26, 2025, as tech stocks faltered and the White House prepared to announce tariffs on car imports. The S&P 500 dropped 1.1%, while the Dow slipped 0.3%. Nvidia (NVDA) stock slipped 5.7%, leading losses on the Dow, amid concerns about additional constraints on AI chip sales in China. The stock had gained more than 27% in the week leading up to Wednesday's session, boosted by the potential for watered-down tariffs and several endorsements. Discount retailer Dollar Tree (DLTR) announced a deal to sell its Family Dollar brand to the private equity firms Brigade Capital Management and Macellum Capital Management for $1 billion. Read the original article on Inves